In [3]:
file_line = 'dom_in_v1.geojson'

import json
import numpy as np
with open(file_line, 'r') as f:
    data = json.load(f)

In [6]:
def get_median(data):
    data = data.replace('(',''); data = data.replace(')','')
    data_all = data.split(',')
    datas = []
    #if sum([len(d) for d in data_all]) == 0:
    #    return 0.
    #else:
    for d in data_all:
        if len(d) > 0: 
            datas.append(float(d)) 
    return np.median(datas)


In [10]:
import re

for feature in data['features']:
    # take middle dip
    dip = feature['properties']['dip']
    if type(dip) is str:
        feature['properties']['dip'] = get_median(dip)
    else:
        feature['properties']['dip'] = 70
    # get sliprates set up
    nsr = feature['properties']['net_slip_rate']
    ssr = feature['properties']['strike_slip_rate']
    shr = feature['properties']['shortening_rate']

    if type(nsr) is str:
        isnum = any(i.isdigit() for i in nsr)
    
    if (type(nsr) is str) & (isnum is True):
    #and (type(ssr) is not str) & (type(shr) is not str):
        feature['properties']['v_rl'] = get_median(nsr)
        feature['properties']['v_ex'] = 0.
        
    elif (type(ssr) is str) & (type(shr) is not str):
        feature['properties']['v_rl'] = get_median(ssr)
        feature['properties']['v_ex'] = 0
    
    elif (type(shr) is str) & (type(ssr) is not str):# & (type(nsr) is not str):
        feature['properties']['v_ex'] = get_median(shr)
        feature['properties']['v_rl'] = 0
        
    elif (type(ssr) is str) & (type(shr) is str):# & (type(nsr) is not str):
        feature['properties']['v_rl'] = get_median(ssr)
        feature['properties']['v_ex'] = get_median(shr)
    else:
        import pdb; pdb.set_trace()
        feature['properties']['v_ex'] = 0
        feature['properties']['v_rl'] = 0
            
    feature['properties']['e_ex'] = 0.
    feature['properties']['e_rl'] = 0.
    feature['properties']['fid'] = feature['properties']['source_id']


In [11]:
file2 = 'dom_in_forced.geojson'
with open(file2, 'w+') as f:
    json.dump(data, f, indent=2)